### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re
import pickle

### Import data

In [2]:
df = pd.read_pickle('../assets/df.pkl')

In [3]:
len(df)

11477723

### Movie

In [4]:
df.head()

,adult,budget,id,original_title,overview,popularity,revenue,runtime,genre_unlisted,userId,...,horror,movie,music,mystery,romance,science,thriller,tv,war,western
0,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",1923.0,...,0,0,0,0,0,0,0,0,0,0
1,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",2103.0,...,0,0,0,0,0,0,0,0,0,0
2,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",5380.0,...,0,0,0,0,0,0,0,0,0,0
3,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",6177.0,...,0,0,0,0,0,0,0,0,0,0
4,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",6525.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.columns

Index(['adult', 'budget', 'id', 'original_title', 'overview', 'popularity',
       'revenue', 'runtime', 'genre_unlisted', 'userId', 'rating', 'action',
       'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'fiction', 'foreign', 'genre_not_specified',
       'history', 'horror', 'movie', 'music', 'mystery', 'romance', 'science',
       'thriller', 'tv', 'war', 'western'],
      dtype='object')

In [6]:
movie_features = ['id', 'original_title', 'adult', 'overview', 'popularity', 
                  'revenue', 'runtime', 'genre_unlisted', 'action', 'adventure',
                  'animation', 'comedy', 'crime', 'documentary', 'drama', 'family',
                  'fantasy', 'fiction', 'foreign', 'genre_not_specified', 'history',
                  'horror', 'movie', 'music', 'mystery', 'romance', 'science',
                  'thriller', 'tv', 'war', 'western']

In [7]:
movie_df = df[movie_features]

In [8]:
movie_df.drop_duplicates(subset = ['id'], inplace = True)

/Users/yoonhokim/miniforge3/envs/tensorflow/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


drop duplicate values

In [9]:
movie_df = movie_df.reset_index().drop(columns = ['index'])

In [10]:
movie_df.head()

,id,original_title,adult,overview,popularity,revenue,runtime,genre_unlisted,action,adventure,...,horror,movie,music,mystery,romance,science,thriller,tv,war,western
0,862,Toy Story,False,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",0,0,...,0,0,0,0,0,0,0,0,0,0
1,8844,Jumanji,False,When siblings Judy and Peter discover an encha...,17.015539,262797249.0,104.0,"Adventure, Fantasy, Family",0,1,...,0,0,0,0,0,0,0,0,0,0
2,15602,Grumpier Old Men,False,A family wedding reignites the ancient feud be...,11.7129,0.0,101.0,"Romance, Comedy",0,0,...,0,0,0,0,1,0,0,0,0,0
3,31357,Waiting to Exhale,False,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156.0,127.0,"Comedy, Drama, Romance",0,0,...,0,0,0,0,1,0,0,0,0,0
4,11862,Father of the Bride Part II,False,Just when George Banks has recovered from his ...,8.387519,76578911.0,106.0,Comedy,0,0,...,0,0,0,0,0,0,0,0,0,0


### User

In [11]:
df.head()

,adult,budget,id,original_title,overview,popularity,revenue,runtime,genre_unlisted,userId,...,horror,movie,music,mystery,romance,science,thriller,tv,war,western
0,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",1923.0,...,0,0,0,0,0,0,0,0,0,0
1,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",2103.0,...,0,0,0,0,0,0,0,0,0,0
2,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",5380.0,...,0,0,0,0,0,0,0,0,0,0
3,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",6177.0,...,0,0,0,0,0,0,0,0,0,0
4,False,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033.0,81.0,"Animation, Comedy, Family",6525.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.columns

Index(['adult', 'budget', 'id', 'original_title', 'overview', 'popularity',
       'revenue', 'runtime', 'genre_unlisted', 'userId', 'rating', 'action',
       'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama',
       'family', 'fantasy', 'fiction', 'foreign', 'genre_not_specified',
       'history', 'horror', 'movie', 'music', 'mystery', 'romance', 'science',
       'thriller', 'tv', 'war', 'western'],
      dtype='object')

In [13]:
user_features = ['id', 'original_title', 'popularity', 'userId', 'rating']

In [14]:
user_df = df[user_features]